### Aim: Implement 2D Kalman Filter using steps mentioned in Theory and calculate position and velocity of object after 6 iterations. 

### Use following data:
x0= 4000 meter,  Vx0= 280 m/sec, y0= 3000 meter, Vy0=120 m/sec,  
Initial Conditions:
ax= 2 m/sec2,
δt= 1; 
Process errors in process covariance matrix:
δPx= 20 m/s
δPvx= 5 m/s
Observation error:
Δx= 25 m, δVx= 6 m/s

Observations:

X1= 4260	V1x= 282 m/s

X2= 4550	V2x= 285 m/s

X3= 4860	V3x= 286 m/s

X4= 5110	V4x= 290 m/s



Process control matrix

Kalman Gain 

Error in Measurement + Prediction ⇒ Predicted value 

Estimated Value by the Kalman Gain

In [ ]:
import numpy as np

In [ ]:
# Initial Data

x0 = 4000 #m
vx0 = 280 #m/s
ax = 2 #m/s2
dt = 1

In [ ]:
# Errors

# Process Errors 
dPx = 20 #m
dPvx = 5 #m/s

# Observation Errors
dx = 25 #m
dVx = 6 #m/s


In [ ]:
# Observations

x1 = 4260 
vx1 = 282 

x2 = 4550
vx2 = 285

x3 = 4860
vx3 = 286

x4 = 5110
vx4 = 290 

### Calcluate Predicted State

In [ ]:
def A(dt):
    return np.array([[1, dt],
                    [0 , 1]])

In [ ]:
def B(dt):
    return np.array([[0.5*(dt**2)],
                        [dt]])

In [ ]:
def R(dx, dVx):
    # Observation errors
    return np.array([[dx**2, 0],
                     [0 , dVx**2]])

In [ ]:
def CalculatePredictedState(X, dt, ax, wk = 0):
    Xkp = np.add(np.dot(A(dt), X),np.dot(B(dt), ax))
    Xkp = np.add(Xkp, wk)
    return Xkp    

In [ ]:
X = np.array([[x0], [vx0]])
X1p = CalculatePredictedState(X, dt, ax)
X1p

array([[4281.],
       [ 282.]])

### Initial Process Covariance Matrix

In [ ]:
def InitalProcessCovMat(dPx, dPvx):
    # we are ignoring the non-diagonal terms
    P0 = np.array([[dPx**2, 0], 
                         [0 , dPvx**2]])
    return P0

In [ ]:
P0 = InitalProcessCovMat(dPx, dPvx)

### Predicted Process Covariance

In [ ]:
def PredictedProcessCovariance(Pkminus1, dt, Qr = 0):

    Pkp = A(dt).dot(Pkminus1).dot(A(dt).transpose()) + Qr
    Pkp = Pkp*np.identity(len(Pkp)) # ignoring non-diagonal elements
    return Pkp

In [ ]:
P1p = PredictedProcessCovariance(P0, dt)
P1p

array([[425.,   0.],
       [  0.,  25.]])

### Kalman Gain

In [ ]:
def KalmanGain(Pkp, dx, dVx):
    H = np.identity(len(Pkp))
    numerator = Pkp*H.transpose()
    d1 = H.dot(Pkp).dot(H.transpose())
    denominator = np.add(d1, R(dx, dVx))
    K = np.divide(numerator, denominator)
    K = np.nan_to_num(K)
    return K

In [ ]:
K1 = KalmanGain(P1, dx, dVx)
K1

<ipython-input-14-538da1436b4e>:6: RuntimeWarning: invalid value encountered in true_divide
  K = np.divide(numerator, denominator)


array([[0.4047619 , 0.        ],
       [0.        , 0.40983607]])

### Finding new Observation

In [ ]:
def newObservation(x1, vx1, Z1=0):
    Ykm = np.array([[x1], [vx1]])
    C = np.identity(len(Ykm))
    Yk = np.dot(C, Ykm) + Z1
    return Yk

In [ ]:
Y1 = newObservation(x1, vx1)
Y1

array([[4260.],
       [ 282.]])

### Calculate Current Estimate

In [ ]:
def CurrentEstimate(Xkp, K, Yk):
    H = np.identity(len(Xkp[0]))
    temp1 = np.subtract(Yk , np.multiply(H,Xkp))
    Xk = np.add(Xkp , np.dot(K, temp1))
    return Xk

In [ ]:
X1 = CurrentEstimate(X1p, K1, Y1)
X1

array([[4272.5],
       [ 282. ]])

### Update Process Covariance Matrix

In [ ]:
def newProcessCovMat(K, Pkp):
    I = H = np.identity(len(Pkp))
    temp = np.subtract(I, np.multiply(K, H))
    Pk = np.dot(temp, Pkp)
    return Pk

In [ ]:
P1 = newProcessCovMat(K1, P1p)
P1

array([[252.97619048,   0.        ],
       [  0.        ,  14.75409836]])

### Creating a Pipeline

In [ ]:
def KalmanFilter(x, vx, dt, ax, dPx, dPvx):

    # Suppress/hide the warning
    np.seterr(invalid='ignore')

    # printing options
    np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

    # Step 0: Initial Values (X0 and P0)
    Xkminus1 = np.array([[x[0]], [vx[0]]]) 
    Pkminus1 = InitalProcessCovMat(dPx, dPvx)

    final_results = []
    
    for k in range(1, len(x)):

        print("--------------------------------------------------------------------------------------")
        print("Iteration with k = ", k)
        print("--------------------------------------------------------------------------------------")

        # Step 1: Calculate Predicted State
        Xkp = CalculatePredictedState(Xkminus1, dt, ax)

        print(f"X{k}p = \n", Xkp, "\n\n")

        # Step 2: Predicted Process Covariance 
        Pkp = PredictedProcessCovariance(Pkminus1, dt) 

        print(f"P{k}p = \n", Pkp, "\n\n")

        # Step 3: Calculate the Kalman Gain
        KG = KalmanGain(Pkp, dx, dVx)

        print(f"K{k} = \n", KG, "\n\n")

        # Step 4: Finding New Observation
        # Here k is taken instead of k+1 because of 0 indexing in the x and vx list
        Yk = newObservation(x[k], vx[k]) 

        print(f"Y{k} = \n", Yk, "\n\n")

        # Step 5: Calculate the current Estimate State
        Xk = CurrentEstimate(Xkp, KG , Yk)
        final_results.append(Xk)

        print(f"X{k} = \n", Xk, "\n\n")

        
        # If not last iteration - update the matrices 
        if (k==len(x)-1):
            break


        # Step 6: Updating Process Covariance Matrix
        Pminus1 = newProcessCovMat(KG, Pkp)

        print("New P = \n", Pminus1, "\n\n")

        # Step 7: Updating State Matrix 

        Xkminus1 = Xk


    return final_results


In [ ]:
x = [x0, x1, x2, x3, x4]
vx = [vx0, vx1, vx2, vx3, vx4]

In [ ]:
final_result = KalmanFilter(x, vx, dt, ax, dPx, dPvx)

--------------------------------------------------------------------------------------
Iteration with k =  1
--------------------------------------------------------------------------------------
X1p = 
 [[ 4281.000]
 [ 282.000]] 


P1p = 
 [[ 425.000  0.000]
 [ 0.000  25.000]] 


K1 = 
 [[ 0.405  0.000]
 [ 0.000  0.410]] 


Y1 = 
 [[ 4260.000]
 [ 282.000]] 


X1 = 
 [[ 4272.500]
 [ 282.000]] 


New P = 
 [[ 252.976  0.000]
 [ 0.000  14.754]] 


--------------------------------------------------------------------------------------
Iteration with k =  2
--------------------------------------------------------------------------------------
X2p = 
 [[ 4555.500]
 [ 284.000]] 


P2p = 
 [[ 425.000  0.000]
 [ 0.000  25.000]] 


K2 = 
 [[ 0.405  0.000]
 [ 0.000  0.410]] 


Y2 = 
 [[ 4550.000]
 [ 285.000]] 


X2 = 
 [[ 4553.274]
 [ 284.410]] 


New P = 
 [[ 252.976  0.000]
 [ 0.000  14.754]] 


--------------------------------------------------------------------------------------
Iteration wit

In [ ]:
for i in range(len(final_result)):
    print(f'X{i+1} = \n', final_result[i], '\n')

X1 = 
 [[ 4272.500]
 [ 282.000]] 

X2 = 
 [[ 4553.274]
 [ 284.410]] 

X3 = 
 [[ 4847.312]
 [ 286.242]] 

X4 = 
 [[ 5124.615]
 [ 288.962]] 



### Conclusion: 
Hence we have thus implemented the Kalman Filter from scratch in Python.
We have obtained the results for X1, X2, X3, X4 which are the estimated States for time instants 1, 2, 3, 4.
